Test

In [ ]:
!wget "https://raw.githubusercontent.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/master/src/datasets/exploration_datasets/gold/qa_dataset_col_mex_news_squad2.json" -O "qa_dataset_col_mex_news_squad2.json"

--2025-10-24 02:02:14--  https://raw.githubusercontent.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/master/src/datasets/exploration_datasets/gold/qa_dataset_col_mex_news_squad2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133 [text/plain]
Saving to: ‘qa_dataset_col_mex_news_squad2.json’

qa_dataset_col_mex_ 100%[===================>]     133  --.-KB/s    in 0s      

2025-10-24 02:02:14 (3.06 MB/s) - ‘qa_dataset_col_mex_news_squad2.json’ saved [133/133]



In [ ]:
!ls -lh "/content/drive/MyDrive/qa_dataset_col_mex_news_squad2.json"
!head -n 10 "/content/drive/MyDrive/qa_dataset_col_mex_news_squad2.json"

-rw------- 1 root root 133 Oct 24 02:01 /content/drive/MyDrive/qa_dataset_col_mex_news_squad2.json
version https://git-lfs.github.com/spec/v1
oid sha256:7103b65b3b45ee4d63d57681fceae6df1123a839f483467f74a63583f3b6e14e
size 20171386


In [1]:
!pip install transformers evaluate torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00


In [3]:
!pip install simpletransformers transformers datasets huggingface_hub scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.8/330.8 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 121.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e4c4b37ac073a3bbd06e43b782520ec8177cb66eddce54c07984beb1cf3aba93
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [4]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import evaluate
import json
from tqdm import tqdm
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from sklearn.model_selection import train_test_split

In [5]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

In [6]:
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

GPU available: True
GPU name: Tesla T4


# Paso 1: Train directly with your data

In [11]:
import json

with open("/content/qa_dataset_col_mex_news_squad2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

print(f"✅ Loaded {len(dataset)} records successfully!")


✅ Loaded 1 records successfully!


In [12]:
dataset = dataset["data"]

In [13]:
# 🔧 1️⃣ Flatten your dataset so each row has 'context' and 'qas'
def flatten_squad(dataset):
    new_data = []
    for article in dataset:
        for para in article["paragraphs"]:
            new_data.append({
                "context": para["context"],
                "qas": para["qas"]
            })
    return new_data

flattened_data = flatten_squad(dataset)

# 🔧 2️⃣ Split into train and eval sets (70/30)
train_data, eval_data = train_test_split(flattened_data, test_size=0.3, random_state=42)

In [14]:
print(f"✅ Training samples: {len(train_data)}")
print(f"✅ Eval samples: {len(eval_data)}")

✅ Training samples: 2273
✅ Eval samples: 975


# Paso 2: Define hyperparameters

In [15]:
model_args = QuestionAnsweringArgs()

# Training behavior
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 4
model_args.learning_rate = 2e-5
model_args.gradient_accumulation_steps = 1
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 500
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.save_steps = -1
model_args.best_model_dir = "./outputs/best_model/"
model_args.output_dir = "./outputs/"

# Optimization
model_args.max_seq_length = 384
model_args.doc_stride = 128
model_args.warmup_ratio = 0.1
model_args.max_answer_length = 30

# Logging
model_args.logging_steps = 100
model_args.evaluate_during_training_verbose = True
model_args.manual_seed = 42

# Resource handling
model_args.use_multiprocessing = False  # safer for notebooks
model_args.fp16 = torch.cuda.is_available()  # use mixed precision if CUDA available

# Paso 3: Load model

In [16]:
model = QuestionAnsweringModel(
    model_type="bert",
    model_name="mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",  # BETO distilled
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

The following layers were not sharded: bert.encoder.layer.*.output.dense.weight, qa_outputs.bias, qa_outputs.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.embeddings.token_type_embeddings.weight, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.intermediate.dense.bias, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.attention.self.value.bias, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.attention.self.key.bias, bert.encoder.layer.*.attention.self.query.weight, bert.encoder.layer.*.attention.self.key.weight, bert.encode

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Paso 4: TRAIN THE MODEL

In [17]:
model.train_model(train_data, eval_data=eval_data)

add example index and unique id: 100%|██████████| 14684/14684 [00:00<00:00, 735858.63it/s]


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:697: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 4:   0%|          | 0/8646 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:720: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


convert squad examples to features:   0%|          | 0/6282 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/6282 [00:19<34:17:08, 19.65s/it]

convert squad examples to features:   8%|▊         | 524/6282 [00:20<02:39, 36.18it/s] 

convert squad examples to features:  17%|█▋        | 1047/6282 [00:25<01:30, 57.54it/s]

convert squad examples to features:  25%|██▍       | 1570/6282 [00:33<01:16, 61.29it/s]

convert squad examples to features:  50%|████▉     | 3139/6282 [00:39<00:25, 124.92it/s]

convert squad examples to features:  58%|█████▊    | 3662/6282 [00:45<00:23, 110.16it/s]

convert squad examples to features: 100%|██████████| 6282/6282 [00:50<00:00, 123.93it/s]


add example index and unique id: 100

...



convert squad examples to features:   0%|          | 0/6282 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/6282 [00:23<40:53:08, 23.43s/it]

convert squad examples to features:   8%|▊         | 524/6282 [00:24<03:12, 29.93it/s] 

convert squad examples to features:   9%|▉         | 572/6282 [00:24<02:52, 33.17it/s]

convert squad examples to features:  17%|█▋        | 1047/6282 [00:25<01:05, 79.74it/s]

convert squad examples to features:  25%|██▍       | 1570/6282 [00:32<00:58, 80.30it/s]

convert squad examples to features:  50%|████▉     | 3139/6282 [00:35<00:17, 180.32it/s]

convert squad examples to features:  58%|█████▊    | 3662/6282 [00:44<00:22, 118.74it/s]

convert squad examples to features: 100%|██████████| 6282/6282 [00:50<00:00, 124.09it/s]


add example index and unique id: 100%|██████████| 6282/6282 [00:00<00:00, 441483.49it/s]


Running Evaluation:   0%|          | 0/3883 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 6282/6282 [00:51<00:00, 122.51it/s]

add example index and unique id: 100%|██████████| 6282/6282 [00:00<00:00, 376591.74it/s]


Running Evaluation:   0%|          | 0/3883 [00:00<?, ?it/s]

(34584,
 {'global_step': [500,
   1000,
   1500,
   2000,
   2500,
   3000,
   3500,
   4000,
   4500,
   5000,
   5500,
   6000,
   6500,
   7000,
   7500,
   8000,
   8500,
   8646,
   9000,
   9500,
   10000,
   10500,
   11000,
   11500,
   12000,
   12500,
   13000,
   13500,
   14000,
   14500,
   15000,
   15500,
   16000,
   16500,
   17000,
   17292,
   17500,
   18000,
   18500,
   19000,
   19500,
   20000,
   20500,
   21000,
   21500,
   22000,
   22500,
   23000,
   23500,
   24000,
   24500,
   25000,
   25500,
   25938,
   26000,
   26500,
   27000,
   27500,
   28000,
   28500,
   29000,
   29500,
   30000,
   30500,
   31000,
   31500,
   32000,
   32500,
   33000,
   33500,
   34000,
   34500,
   34584],
  'correct': [3416,
   3669,
   3949,
   4009,
   4051,
   4206,
   4099,
   4152,
   4237,
   4239,
   4213,
   4257,
   4269,
   4215,
   4214,
   4143,
   4154,
   4278,
   4219,
   4257,
   4298,
   4270,
   4177,
   4192,
   4182,
   4210,
   4294,
   4268,
   4

# Paso 5: EVALUATE

In [24]:
result, texts = model.eval_model(eval_data)
print("📊 Evaluation results:")
print(result)

add example index and unique id: 100%|██████████| 6282/6282 [00:00<00:00, 434186.66it/s]


Running Evaluation:   0%|          | 0/3883 [00:00<?, ?it/s]

📊 Evaluation results:
{'correct': 4344, 'similar': 1605, 'incorrect': 333, 'eval_loss': -9.376533125160957}


In [26]:
print(type(texts))

<class 'dict'>


In [27]:
print(type(texts))
print(len(texts))


<class 'dict'>
3


In [28]:
print(texts)

{'correct_text': {'qa-21210': 'Alejandro Gertz Manero', 'qa-21212': '100 mil pesos', 'qa-21213': '', 'qa-2102': 'Museo Edgar Negret', 'qa-2105': 'David Manzur, Fernando Dávila, y Miguel Moyano', 'qa-2107': '', 'qa-22671': 'Ricardo Sánchez Pérez del Pozo', 'qa-22672': 'Sara Irene Herrerías Guerra', 'qa-22673': 'Fundación para la Justicia', 'qa-22674': 'fiscal especial para la Atención de Delitos Cometidos contra la Libertad de Expresión', 'qa-22675': 'Maestría en Derecho Internacional y Derechos Humanos', 'qa-22676': '', 'qa-22677': '', 'qa-10011': '12 de las 19 localidades', 'qa-10013': '', 'qa-21081': '12 de las 19 localidades', 'qa-21083': '', 'qa-22904': '', 'qa-14741': 'Noam Shomron', 'qa-14742': 'Rare Genomic Institute', 'qa-14745': '', 'qa-21736': 'Claudia Sheinbaum', 'qa-21737': '25 millones de dólares', 'qa-21739': 'Grupo Salinas', 'qa-21741': '', 'qa-21742': '', 'qa-254': 'Blanca Uribe, Helvia Mendoza y Teresita Gómez', 'qa-256': '1538', 'qa-259': '', 'qa-260': '', 'qa-5394': 

In [33]:
import re
from evaluate import load

In [45]:
from evaluate import load
squad_metric = load("squad")

def compute_squad_metrics_from_eval(texts, eval_data):
    preds = []
    refs = []

    for article in eval_data:
        context = article["context"]
        for qa in article["qas"]:
            qid = qa["id"]

            # Retrieve prediction (only the string)
            pred_text = ""
            if qid in texts["correct_text"]:
                pred_text = texts["correct_text"][qid]
            elif qid in texts["similar_text"]:
                pred_text = texts["similar_text"][qid]
            elif qid in texts["incorrect_text"]:
                pred_text = texts["incorrect_text"][qid]

            # If prediction is a dict (contains 'predicted' key), extract that
            if isinstance(pred_text, dict):
                pred_text = pred_text.get("predicted", "")

            preds.append({
                "id": qid,
                "prediction_text": str(pred_text)
            })

            # Ground truth reference
            answers = [a["text"] for a in qa["answers"]] if qa["answers"] else [""]
            answer_starts = [a["answer_start"] for a in qa["answers"]] if qa["answers"] else [0]
            refs.append({
                "id": qid,
                "answers": {"text": answers, "answer_start": answer_starts}
            })

    # ✅ Now compute EM and F1 correctly
    metrics = squad_metric.compute(predictions=preds, references=refs)
    return metrics


In [46]:
eval_data[0]

{'context': '. Ciudad de México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez, implicado en el caso de la red de huachicol fiscal , fue hallado muerto en Tamaulipas. La institución lamentó el deceso, envió sus condolencias a familiares y compañeros, y reiteró su disposición para colaborar en el esclarecimiento de los hechos. SEMARmx La Secretaría de Marina-Armada de México lamenta profundamente el sensible fallecimiento de uno de sus elementos, en el estado de Tamaulipas. La institución extiende su más sentido pésame y solidaridad a su familia, amigos y compañeros, reiterando que se les brindarán todos los apoyos necesarios y los que por ley corresponden. Asimismo, se reitera la colaboración con las autoridades competentes para el pleno esclarecimiento de los hechos, publicó la dependencia. El pasado domingo, el gabinete de seguridad del gobierno federal anunció la detención de 14 personas implicados en la red de huachicol fiscal, entre ellos cinco marinos en a

In [47]:
metrics = compute_squad_metrics_from_eval(texts, eval_data)
print("📊 Base evaluation:", result)
print("🏆 SQuAD-style metrics:", metrics)


📊 Base evaluation: {'correct': 4344, 'similar': 1605, 'incorrect': 333, 'eval_loss': -9.376533125160957}
🏆 SQuAD-style metrics: {'exact_match': 73.73447946513849, 'f1': 53.50684349692086}


In [21]:
from evaluate import load

def compute_squad_metrics(texts):
    # Prepare the data in the format HuggingFace's metric expects
    preds = []
    refs = []
    for i, ex in enumerate(texts):
        preds.append({
            "id": str(i),
            "prediction_text": ex["prediction_text"]
        })
        refs.append({
            "id": str(i),
            "answers": {"text": [ex["answer_text"]], "answer_start": [0]}
        })

    metric = load("squad")
    results = metric.compute(predictions=preds, references=refs)
    return results

metrics = compute_squad_metrics(texts)
print("📊 Evaluation results:")
print("🏆 SQuAD metrics:", metrics)

add example index and unique id: 100%|██████████| 6282/6282 [00:00<00:00, 397516.98it/s]


Running Evaluation:   0%|          | 0/3883 [00:00<?, ?it/s]

TypeError: string indices must be integers, not 'str'

In [37]:
print(type(texts))
print(texts.keys())



<class 'dict'>
dict_keys(['correct_text', 'similar_text', 'incorrect_text'])


In [23]:
# Define your input
to_predict = [
    {
        "context": "Gabriel García Márquez nació en Aracataca, Colombia, en 1927. Fue un famoso escritor colombiano y premio Nobel de Literatura.",
        "qas": [
            {
                "question": "¿Dónde nació Gabriel García Márquez?",
                "id": "1"
            }
        ],
    }
]

# Make prediction
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8848.74it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '1', 'answer': ['Aracataca, Colombia', 'Aracataca', 'Aracataca, Colombia,', 'Colombia', 'Ara', 'Aracataca, Colombia, en 1927.', 'en Aracataca, Colombia', 'Aracataca,', 'Aracat', 'aca, Colombia', 'cataca, Colombia', ', Colombia', 'Aracataca, Colombia, en', 'nació en Aracataca, Colombia', 'Gabriel García Márquez nació en Aracataca, Colombia', 'Aracataca, Colombia, en 1927. Fue un famoso escritor colombiano', 'Colombia,', ',', '']}]


/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:1358: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [50]:
from evaluate import load
squad_metric = load("squad")

# 1️⃣ Run model predictions
predictions, raw_outputs = model.predict(eval_data)

# 2️⃣ Convert predictions and references into SQuAD format
preds = []
refs = []

for article, pred_list in zip(eval_data, predictions):
    for qa, pred in zip(article["qas"], pred_list):
        qid = qa["id"]

        # Prediction from model
        prediction_text = pred.get("answer", "") if isinstance(pred, dict) else ""
        preds.append({"id": qid, "prediction_text": prediction_text})

        # Ground truth
        answers = [a["text"] for a in qa["answers"]] if qa["answers"] else [""]
        answer_starts = [a["answer_start"] for a in qa["answers"]] if qa["answers"] else [0]
        refs.append({
            "id": qid,
            "answers": {"text": answers, "answer_start": answer_starts}
        })

# 3️⃣ Compute EM and F1
metrics = squad_metric.compute(predictions=preds, references=refs)
print("🏆 Hugging Face metrics:", metrics)


add example index and unique id: 100%|██████████| 6282/6282 [00:00<00:00, 427210.22it/s]


Running Prediction:   0%|          | 0/3883 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:1358: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


🏆 Hugging Face metrics: {'exact_match': 100.0, 'f1': 0.0}


In [51]:
from evaluate import load
import re

# Load Hugging Face SQuAD metric
squad_metric = load("squad")

def normalize_text(s):
    """Lowercase, remove extra whitespace and punctuation for F1 calculation."""
    s = s or ""
    s = s.lower()
    s = re.sub(r"[.,;!?¿¡\"']", " ", s)
    s = " ".join(s.split())
    return s

def compute_squad_metrics_safe(texts, eval_data):
    preds = []
    refs = []

    for article in eval_data:
        for qa in article["qas"]:
            qid = qa["id"]

            # 1️⃣ Get prediction
            pred_text = ""
            if qid in texts.get("correct_text", {}):
                pred_text = texts["correct_text"][qid]
            elif qid in texts.get("similar_text", {}):
                pred_text = texts["similar_text"][qid]
            elif qid in texts.get("incorrect_text", {}):
                pred_text = texts["incorrect_text"][qid]

            # If prediction is a dict (from previous outputs), extract only 'predicted'
            if isinstance(pred_text, dict):
                pred_text = pred_text.get("predicted", "")

            pred_text = normalize_text(pred_text)

            # 2️⃣ Get reference
            if qa.get("answers"):
                answers = [normalize_text(a["text"]) for a in qa["answers"]]
                answer_starts = [a["answer_start"] for a in qa["answers"]]
            else:
                answers = [""]
                answer_starts = [0]

            preds.append({"id": qid, "prediction_text": pred_text})
            refs.append({"id": qid, "answers": {"text": answers, "answer_start": answer_starts}})

    print(f"✅ Evaluating {len(preds)} QA pairs")
    metrics = squad_metric.compute(predictions=preds, references=refs)
    return metrics


In [54]:
# `result` is your simpletransformers base evaluation
result, texts = model.eval_model(eval_data)

# Compute Hugging Face SQuAD metrics safely
metrics = compute_squad_metrics_safe(texts, eval_data)

print("📊 Base evaluation:", result)
print("🏆 Hugging Face SQuAD metrics:", metrics)


add example index and unique id: 100%|██████████| 6282/6282 [00:00<00:00, 406363.63it/s]


Running Evaluation:   0%|          | 0/3883 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:1194: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


✅ Evaluating 6282 QA pairs
📊 Base evaluation: {'correct': 2145, 'similar': 4137, 'incorrect': 0, 'eval_loss': -9.376533125160957}
🏆 Hugging Face SQuAD metrics: {'exact_match': 34.14517669531996, 'f1': 0.0}


# Paso 6:

In [97]:
import os

# Folder to save
local_path = "./QA_model"
os.makedirs(local_path, exist_ok=True)

# Save the Hugging Face model & tokenizer directly
model.model.save_pretrained(local_path)       # Saves weights + config
model.tokenizer.save_pretrained(local_path)   # Saves vocab + tokenizer config

# Check files
!ls -l ./QA_model


total 427080
-rw-r--r-- 1 root root       615 Oct 25 14:27 config.json
-rw-r--r-- 1 root root 437070648 Oct 25 14:27 model.safetensors
-rw-r--r-- 1 root root       125 Oct 25 14:27 special_tokens_map.json
-rw-r--r-- 1 root root      1293 Oct 25 14:27 tokenizer_config.json
-rw-r--r-- 1 root root    242349 Oct 25 14:27 vocab.txt


In [98]:
import shutil

shutil.make_archive("QA_model", 'zip', local_path)
print("✅ Zipped model")
!ls -lh QA_model.zip

✅ Zipped model
-rw-r--r-- 1 root root 387M Oct 25 14:28 QA_model.zip


In [99]:
from google.colab import files
files.download("QA_model.zip")  # download to your computer

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [100]:
# Or move to Drive
import shutil
shutil.move("QA_model.zip", "/content/drive/MyDrive/Thesis_QA_Optimization/Model")


'/content/drive/MyDrive/Thesis_QA_Optimization/Model/QA_model.zip'

In [101]:
from simpletransformers.question_answering import QuestionAnsweringModel

# Path to the folder containing the saved model
model_path = "./QA_model"  # change if different

# Reload the model
my_model = QuestionAnsweringModel(
    "bert",
    model_path,
    use_cuda=True  # set to False if no GPU
)

The following layers were not sharded: bert.encoder.layer.*.output.dense.weight, qa_outputs.bias, qa_outputs.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.embeddings.token_type_embeddings.weight, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.intermediate.dense.bias, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.attention.self.value.bias, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.attention.self.key.bias, bert.encoder.layer.*.attention.self.query.weight, bert.encoder.layer.*.attention.self.key.weight, bert.encode

In [104]:
# Context & question
context = "Ciudad de México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez, fue hallado muerto en Tamaulipas."
question = "¿Quién fue hallado muerto en Tamaulipas?"

# Prepare input in SimpleTransformers format
to_predict = [
    {
        "context": context,
        "qas": [
            {
                "id": "0",
                "question": question,
                "answers": [{"text": " ", "answer_start": 0}],
                "is_impossible": False
            }
        ]
    }
]

# Run prediction
answers = my_model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 12052.60it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0', 'answer': ['Abraham Jeremías Pérez Ramírez,', 'Abraham', 'El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Ramírez', 'capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez', 'Abraham Jeremías Pérez Ramírez, fue hallado muerto', 'Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado muerto en Tamaulipas.', 'Pérez Ramírez', 'Ciudad de México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías', 'Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado muerto en', 'Abraham Jeremías Pérez Ramírez, fue', 'México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado', '', 'El capitán de la Secretaría de Marina, Abraham']}], [{'id': '0', 'probability': [0.9999867335032386, 3.3964265149031443e-06, 3.065420614261552e-06, 2.982736123103299e-06, 6.05979598924565e-07, 5.

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:1358: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [105]:
# Run prediction
answers = model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8756.38it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0', 'answer': ['Abraham Jeremías Pérez Ramírez,', 'Abraham', 'El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Ramírez', 'capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez', 'Abraham Jeremías Pérez Ramírez, fue hallado muerto', 'Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado muerto en Tamaulipas.', 'Pérez Ramírez', 'Ciudad de México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías', 'Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado muerto en', 'Abraham Jeremías Pérez Ramírez, fue', 'México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado', '', 'El capitán de la Secretaría de Marina, Abraham']}], [{'id': '0', 'probability': [0.9999867373084392, 3.393111324061626e-06, 3.065420625926247e-06, 2.9827361344533594e-06, 6.036171106566265e-07, 5

# Evaluation

In [67]:
!pip install --upgrade huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 41.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.35.3
    Uninstalling huggingface-hub-0.35.3:
      Successfully uninstalled huggingface-hub-0.35.3


In [92]:
from huggingface_hub import notebook_login
notebook_login()

In [93]:
from simpletransformers.question_answering import QuestionAnsweringModel

# Make sure your model is trained/fine-tuned
hf_local_path = "./hf_qa_model"

# Save model and tokenizer
model.model.save_pretrained(hf_local_path)
model.tokenizer.save_pretrained(hf_local_path)
print("✅ Model saved locally in Hugging Face format")


✅ Model saved locally in Hugging Face format


In [94]:
from huggingface_hub import HfApi, Repository

api = HfApi()
repo_name = "qa-spanish-model"
username = "BlueAutomata"
repo_id = f"{username}/{repo_name}"

In [71]:




# Save model in HF format
hf_local_path = "./hf_qa_model"
model.model.save_pretrained(hf_local_path)
model.tokenizer.save_pretrained(hf_local_path)

# Push to hub
repo_url = f"https://huggingface.co/{username}/{repo_name}"
repo = Repository(local_dir=hf_local_path, clone_from=repo_url)
repo.push_to_hub()
print(f"✅ Model pushed to Hugging Face Hub: {repo_url}")


TypeError: HfApi.create_repo() got an unexpected keyword argument 'name'

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel

# Make sure your model is trained/fine-tuned
hf_local_path = "./hf_qa_model"

# Save model and tokenizer
model.model.save_pretrained(hf_local_path)
model.tokenizer.save_pretrained(hf_local_path)
print("✅ Model saved locally in Hugging Face format")

In [66]:
import os
import shutil
from simpletransformers.question_answering import QuestionAnsweringModel

# 1️⃣ Define a local folder in Colab
local_path = "./QA_model"
os.makedirs(local_path, exist_ok=True)

# 2️⃣ Save the model locally (works 100%)
model.save_model(local_path)
print(f"✅ Model saved locally at {local_path}")

# 3️⃣ Zip the folder for easy download
shutil.make_archive("QA_model", 'zip', local_path)
print("✅ Model zipped as QA_model.zip")

# 4️⃣ Download the zip to your computer
from google.colab import files
files.download("QA_model.zip")


✅ Model saved locally at ./QA_model
✅ Model zipped as QA_model.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [62]:
import os

drive_model_path = "/content/drive/MyDrive/Thesis_QA_Optimization/Model"
os.makedirs(drive_model_path, exist_ok=True)

In [63]:
model.save_model(drive_model_path)
print(f"✅ Model saved to Google Drive at {drive_model_path}")

✅ Model saved to Google Drive at /content/drive/MyDrive/Thesis_QA_Optimization/Model


In [57]:
import os

os.makedirs("./outputs/final_model/", exist_ok=True)
model.save_model("./outputs/final_model/")
print("✅ Model saved at ./outputs/final_model/")


✅ Model saved at ./outputs/final_model/


In [58]:
from google.colab import files
!zip -r final_model.zip ./outputs/final_model/
files.download("final_model.zip")

updating: outputs/final_model/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
# Save model and tokenizer to a folder
model.save_model("outputs/final_model")
print("✅ Model saved at outputs/final_model")

✅ Model saved at outputs/final_model


In [9]:
#model_name = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
model_name = "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"

qa_pipeline = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=model_name
)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [56]:
contexto = """
Gabriel García Márquez fue un novelista colombiano, ganador del Premio Nobel de Literatura en 1982.
Es conocido por obras como 'Cien años de soledad' y 'El amor en los tiempos del cólera'.
"""

pregunta = "¿Quién ganó el Premio Nobel de Literatura en 1982?"

resultado = qa_pipeline({
    "context": contexto,
    "question": pregunta
})

print(resultado)

NameError: name 'qa_pipeline' is not defined

In [ ]:
from tqdm import tqdm

predictions = []
references = []

for article in tqdm(dataset["data"]):
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            question = qa["question"]

            # Model prediction
            pred = qa_pipeline(question=question, context=context)

            predictions.append({
                "id": qa["id"],
                "prediction_text": pred.get("answer", ""),
                "no_answer_probability": 1 - pred.get("score", 0)
            })

            # Handle multiple gold answers if present
            gold_texts = [ans["text"] for ans in qa["answers"]]
            gold_starts = [ans.get("answer_start", 0) for ans in qa["answers"]]

            # ✅ FIX: include both "text" and "answer_start"
            references.append({
                "id": qa["id"],
                "answers": {
                    "text": gold_texts,
                    "answer_start": gold_starts
                }
            })



100%|██████████| 3248/3248 [13:49<00:00,  3.91it/s]


In [ ]:
metric = evaluate.load("squad_v2")

In [ ]:
references

[{'id': 'qa-17811', 'answers': {'text': ['Illinois'], 'answer_start': [153]}},
 {'id': 'qa-17812',
  'answers': {'text': ['Tuxedomoon'], 'answer_start': [278]}},
 {'id': 'qa-17813',
  'answers': {'text': ['Qué Viva México'], 'answer_start': [2502]}},
 {'id': 'qa-17814', 'answers': {'text': ['1993'], 'answer_start': [626]}},
 {'id': 'qa-17815',
  'answers': {'text': ['fiesta de la Candelaria'], 'answer_start': [2698]}},
 {'id': 'qa-17816', 'answers': {'text': [], 'answer_start': []}},
 {'id': 'qa-17817', 'answers': {'text': [], 'answer_start': []}},
 {'id': 'qa-17778',
  'answers': {'text': ['La funcionaria'], 'answer_start': [2]}},
 {'id': 'qa-17779',
  'answers': {'text': ['19 por ciento'], 'answer_start': [478]}},
 {'id': 'qa-17781',
  'answers': {'text': ['73.7 por ciento'], 'answer_start': [1730]}},
 {'id': 'qa-17782', 'answers': {'text': [], 'answer_start': []}},
 {'id': 'qa-17938',
  'answers': {'text': ['Chimalistac'], 'answer_start': [363]}},
 {'id': 'qa-17939',
  'answers': {'

In [ ]:
results = metric.compute(predictions=predictions, references=references)

In [ ]:
metric = evaluate.load("squad_v2")
results = metric.compute(predictions=predictions, references=references)
print(results)

{'exact': 40.33673566727082, 'f1': 52.04827644347888, 'total': 20966, 'HasAns_exact': 58.15441783649876, 'HasAns_f1': 75.0512086370753, 'HasAns_total': 14532, 'NoAns_exact': 0.09325458501709667, 'NoAns_f1': 0.09325458501709667, 'NoAns_total': 6434, 'best_exact': 55.33244300295717, 'best_exact_thresh': 0.6415619850158691, 'best_f1': 64.63131456858878, 'best_f1_thresh': 0.7467862963676453}


In [ ]:
print("📊 Evaluation Results:")
print(f"Exact Match (EM): {results['exact']:.2f}%")
print(f"F1 Score: {results['f1']:.2f}%")
print(f"Total Examples: {results['total']}")
print(f"Has Answer (EM): {results.get('HasAns_exact', 0):.2f}%")
print(f"No Answer (EM): {results.get('NoAns_exact', 0):.2f}%")


📊 Evaluation Results:
Exact Match (EM): 40.34%
F1 Score: 52.05%
Total Examples: 20966
Has Answer (EM): 58.15%
No Answer (EM): 0.09%
